In [2]:
#GPU count and name
!nvidia-smi -L

GPU 0: GeForce RTX 2080 Ti (UUID: GPU-616398ce-ae25-c52b-7c47-f658d30d9d86)


In [107]:
!pip install tensorflow-gpu==2.2.0

In [2]:
!pip install Keras==2.3.1

In [1]:
!python -c 'import keras; print(keras.__version__)'

Using TensorFlow backend.
2.3.1


## Check directories and files

In [1]:
import os
base_dir = os.getcwd()
base_dir

'/home/aikauel/enap'

In [2]:
%ls

07082020.zip
'AerialNet - ONNX.ipynb'
 aerialnet_project/
 alertas_07-08-2020.json
 data/
 Enap_Dataset_Formatting_11_Classes.ipynb
 Enap_Dataset_Formatting_4_Classes.ipynb
 Enap_Dataset_Formatting_7_Classes.ipynb
 Enap_Dataset_Formatting_8_Classes.ipynb
 Enap_FULL_11_classes.ipynb
 Enap_FULL_with_animal.ipynb
 Format_7_classes_as_VIA.ipynb
 Generate_predictions.ipynb
 ije_retinanet/
 images_subset/
 imgs_out.json
 keras-retinanet/
 LOG_FILE.log
 Package_retinanet_multi_object_detector-Copy1.ipynb
 pil_tmp_img.jpg
 predictions_07-08-2020.json
 RetinaNet_11_classes.ipynb
 RetinaNet_4_classes.ipynb
 RetinaNet_7_classes.ipynb
 RetinaNet_8_classes.ipynb
 saved_models/
 SkynetV3-release-2020-07-07/
 SkynetV3-release-2020-07-07.zip
 TFServing.ipynb
'The Effects of Super-Resolution on Object Detection Performance in SatelliteImagery.pdf'
'Traning history.txt'
'Visualización y análisis del Dataset.ipynb'
 Visualization.ipynb
 xview_exploration/


In [3]:
TRAIN_ANNOTATIONS = 'aerialnet_project/11_classes/data/11_classes_dataset_train.csv'
!wc -l {TRAIN_ANNOTATIONS}

12915 aerialnet_project/11_classes/data/11_classes_dataset_train.csv


In [4]:
VAL_ANNOTATIONS = 'aerialnet_project/11_classes/data/11_classes_dataset_test.csv'
!wc -l {VAL_ANNOTATIONS}

511 aerialnet_project/11_classes/data/11_classes_dataset_test.csv


In [5]:
CLASSES_FILE = 'aerialnet_project/11_classes/data/classes.csv'

## Set pretrained model

In [6]:
# utilize best weights from 4 classes model as baseline
PRETRAINED_MODEL = 'aerialnet_project/8_classes/snapshots_alt/resnet50_csv_44.h5'
BATCH_SIZE = 8

# Compute best anchors for dataset annotations

In [7]:
!anchor-optimization {'aerialnet_project/11_classes/data/11_classes_dataset_train.csv'} --no-resize --ratios=5 --threads 100

Using TensorFlow backend.
Optimization ended successfully!

Final best anchor configuration
State: 0.14179
Ratios: [0.382, 0.65, 1.0, 1.538, 2.615]
Scales: [0.491, 0.783, 1.24]
Number of labels that don't have any matching anchor: 25


Save anchor optimization to config.ini file

# Set number of steps per epoch

In [2]:
import pandas as pd
import math
# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

NameError: name 'TRAIN_ANNOTATIONS' is not defined

# Train model

In [9]:
CONFIG_FILE = '11_classes/config.ini'
TENSORBOARD_LOGS_DIR = '11_classes/logs'

### FCNs training (regression, classification) with freezed backbone  

First, train the model freezing the backbone. The backbone weights will be the weights from the 7-classes model train before.
Set gamma with high value (>2.0) to weight the loss of well-classified classes down, forcing the model to learn on harder (in this case: less examples) classes.

In [10]:
!python keras-retinanet/keras_retinanet/bin/train.py \
--config aerialnet_project/11_classes/config.ini \
--freeze-backbone \
--random-transform \
--weights {PRETRAINED_MODEL} \
--weighted-average \
--batch-size 8 \
--steps 291 \
--epochs 50 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \12
--workers 2 \
--tensorboard-dir aerialnet_project/11_classes/logs \
--snapshot-path aerialnet_project/11_classes/snapshots \
--validation-freq 5 \
--gamma 4.0 \
csv 'aerialnet_project/11_classes/data/11_classes_dataset_train.csv' 'aerialnet_project/11_classes/data/classes.csv' \
--val-annotations 'aerialnet_project/11_classes/data/11_classes_dataset_test.csv'

.4778 - val_regression_loss: 1.3160 - val_classification_loss: 0.1056
Running network: 100% (108 of 108) |#####| Elapsed Time: 0:00:11 Time:  0:00:11
Parsing annotations: 100% (108 of 108) |#| Elapsed Time: 0:00:00 Time:  0:00:00
returning metrics...
215 instances of class 0 with average precision: 0.8306 and recall: 0.8910
105 instances of class 1 with average precision: 0.7785 and recall: 0.8629
38 instances of class 2 with average precision: 0.6369 and recall: 0.8507
30 instances of class 3 with average precision: 0.5981 and recall: 0.8722
45 instances of class 4 with average precision: 0.4321 and recall: 0.8084
7 instances of class 5 with average precision: 0.8332 and recall: 0.9637
14 instances of class 6 with average precision: 0.8948 and recall: 0.9157
13 instances of class 7 with average precision: 0.1715 and recall: 0.4089
16 instances of class 8 with average precision: 0.0387 and recall: 0.5245
1 instances of class 9 with average precision: 0.0000 and recall: 0.0000
27 instan

#### ...resume training with new parameters

In [40]:
last_model = 'aerialnet_project/8_classes/snapshots/resnet50_csv_08.h5'

In [42]:
'''!python keras-retinanet/keras_retinanet/bin/train.py \
--config aerialnet_project/8_classes/config.ini \
--freeze-backbone \
--random-transform \
--weights {last_model} \
--initial-epoch 8 \
--weighted-average \
--batch-size 8 \
--steps 369 \
--epochs 100 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir aerialnet_project/8_classes/logs \
--snapshot-path aerialnet_project/8_classes/snapshots \
--validation-freq 5 \
--gamma 4.0 \
csv 'aerialnet_project/8_classes/data/8_classes_dataset_train.csv' 'aerialnet_project/8_classes/data/classes.csv' \
--val-annotations 'aerialnet_project/8_classes/data/8_classes_dataset_test.csv''''

Using TensorFlow backend.
Creating model, this may take a second...
2020-07-31 00:20:38.261859: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-31 00:20:38.290991: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-31 00:20:38.291332: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-07-31 00:20:38.291478: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-31 00:20:38.292410: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.s

tracking <tf.Variable 'Variable:0' shape=(15, 4) dtype=float32, numpy=
array([[ -93.63851 ,  -42.69916 ,   93.63851 ,   42.69916 ],
       [-236.93954 , -108.04443 ,  236.93954 ,  108.04443 ],
       [-300.06024 , -136.82747 ,  300.06024 ,  136.82747 ],
       [ -76.12238 ,  -52.52444 ,   76.12238 ,   52.52444 ],
       [-192.61737 , -132.90598 ,  192.61737 ,  132.90598 ],
       [-243.93062 , -168.31213 ,  243.93062 ,  168.31213 ],
       [ -63.232   ,  -63.232   ,   63.232   ,   63.232   ],
       [-160.      , -160.      ,  160.      ,  160.      ],
       [-202.624   , -202.624   ,  202.624   ,  202.624   ],
       [ -52.511314,  -76.14141 ,   52.511314,   76.14141 ],
       [-132.87277 , -192.66551 ,  132.87277 ,  192.66551 ],
       [-168.27007 , -243.99161 ,  168.27007 ,  243.99161 ],
       [ -42.67953 ,  -93.68157 ,   42.67953 ,   93.68157 ],
       [-107.994774, -237.04851 ,  107.994774,  237.04851 ],
       [-136.76457 , -300.19824 ,  136.76457 ,  300.19824 ]],
      dtype=f

Epoch 9/100
2020-07-31 00:20:47.628695: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-07-31 00:20:49.105020: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
  1/369 [..............................] - ETA: 1:01:09 - loss: 1.0406 - regression_loss: 0.9516 - classification_loss: 0.08892020-07-31 00:20:52.616032: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session started.
2020-07-31 00:20:52.616084: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2020-07-31 00:20:52.616093: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error 

returning metrics...
318 instances of class 0 with average precision: 0.5553 and recall: 0.8713
367 instances of class 1 with average precision: 0.4483 and recall: 0.6853
107 instances of class 2 with average precision: 0.3690 and recall: 0.7652
188 instances of class 3 with average precision: 0.1380 and recall: 0.3841
291 instances of class 4 with average precision: 0.2168 and recall: 0.3939
13 instances of class 5 with average precision: 0.2075 and recall: 0.4508
21 instances of class 6 with average precision: 0.3401 and recall: 0.6596
mAP: 0.3674

Epoch 00014: saving model to aerialnet/7_classes/snapshots/resnet50_csv_14.h5
Epoch 15/100
369/369 [==============================] - 269s 728ms/step - loss: 1.1720 - regression_loss: 1.0721 - classification_loss: 0.0998 - val_loss: 1.5475 - val_regression_loss: 1.4409 - val_classification_loss: 0.1271
Running network: 100% (200 of 200) |#####| Elapsed Time: 0:00:20 Time:  0:00:20
Parsing annotations: 100% (200 of 200) |#| Elapsed Time: 0:

Epoch 23/100
369/369 [==============================] - 252s 682ms/step - loss: 1.1370 - regression_loss: 1.0416 - classification_loss: 0.0954 - val_loss: 1.4482 - val_regression_loss: 1.3515 - val_classification_loss: 0.1269
Running network: 100% (200 of 200) |#####| Elapsed Time: 0:00:20 Time:  0:00:20
Parsing annotations: 100% (200 of 200) |#| Elapsed Time: 0:00:00 Time:  0:00:00
returning metrics...
318 instances of class 0 with average precision: 0.5910 and recall: 0.8897
367 instances of class 1 with average precision: 0.5033 and recall: 0.7247
107 instances of class 2 with average precision: 0.4320 and recall: 0.8036
188 instances of class 3 with average precision: 0.1508 and recall: 0.3720
291 instances of class 4 with average precision: 0.2435 and recall: 0.4359
13 instances of class 5 with average precision: 0.2299 and recall: 0.4542
21 instances of class 6 with average precision: 0.3744 and recall: 0.6586
mAP: 0.4053

Epoch 00023: saving model to aerialnet/7_classes/snapshot

#### Full model training

With a decent mAP result in previous step, we now unfreeze the backbone weights to train the full model.

In [23]:
BATCH_SIZE = 4

# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 2294
Count of annotations: 12915
Number of steps per epoch: 574


In [8]:
last_model = 'aerialnet_project/11_classes/snapshots/resnet50_csv_08.h5'

!python keras-retinanet/keras_retinanet/bin/train.py \
--config aerialnet_project/11_classes/config.ini \
--random-transform \
--weights {last_model} \
--initial-epoch 9 \
--weighted-average \
--batch-size 4 \
--steps 574 \
--epochs 100 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir aerialnet_project/11_classes/logs \
--snapshot-path aerialnet_project/11_classes/snapshots \
--validation-freq 5 \
--lr 1e-5 \
--reduce-lr-patience 2 \
--reduce-lr-factor 0.1 \
--gamma 4.0 \
csv 'aerialnet_project/11_classes/data/11_classes_dataset_train.csv' 'aerialnet_project/11_classes/data/classes.csv' \
--val-annotations 'aerialnet_project/11_classes/data/11_classes_dataset_test.csv'

Using TensorFlow backend.
Creating model, this may take a second...
2020-08-12 17:56:37.381234: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-12 17:56:37.411726: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-12 17:56:37.412021: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-08-12 17:56:37.417601: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-08-12 17:56:37.478857: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.s

## Continue training full model

In [13]:
BATCH_SIZE = 4

# compute number of annotations to compute number of steps per epoch
df = pd.read_csv('aerialnet_project/11_classes/data/extra_dataset_train.csv', header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 216
Count of annotations: 802
Number of steps per epoch: 54


In [14]:
last_model = 'aerialnet_project/11_classes/snapshots/resnet50_csv_14.h5'

!python keras-retinanet/keras_retinanet/bin/train.py \
--config aerialnet_project/11_classes/config.ini \
--random-transform \
--weights {last_model} \
--initial-epoch 14 \
--weighted-average \
--batch-size 4 \
--steps 54 \
--epochs 100 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir aerialnet_project/11_classes/logs \
--snapshot-path aerialnet_project/11_classes/snapshots \
--validation-freq 5 \
--lr 1e-5 \
--reduce-lr-patience 5 \
--reduce-lr-factor 0.1 \
--gamma 4.0 \
csv 'aerialnet_project/11_classes/data/extra_dataset_train.csv' 'aerialnet_project/11_classes/data/classes.csv' \
--val-annotations 'aerialnet_project/11_classes/data/11_classes_dataset_test.csv'

Using TensorFlow backend.
Creating model, this may take a second...
2020-08-12 18:35:22.317509: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-12 18:35:22.344177: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-12 18:35:22.344605: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-08-12 18:35:22.344787: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-08-12 18:35:22.345732: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.s

## Continue training full model 2

In [28]:
last_model = 'aerialnet_project/11_classes/snapshots/resnet50_csv_28.h5'

!python keras-retinanet/keras_retinanet/bin/train.py \
--config aerialnet_project/11_classes/config.ini \
--random-transform \
--weights {last_model} \
--initial-epoch 28 \
--weighted-average \
--batch-size 4 \
--steps 574 \
--epochs 100 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir aerialnet_project/11_classes/logs \
--snapshot-path aerialnet_project/11_classes/snapshots \
--validation-freq 5 \
--lr 1e-5 \
--reduce-lr-patience 5 \
--reduce-lr-factor 0.1 \
--gamma 3.0 \
csv 'aerialnet_project/11_classes/data/11_classes_dataset_train.csv' 'aerialnet_project/11_classes/data/classes.csv' \
--val-annotations 'aerialnet_project/11_classes/data/11_classes_dataset_test.csv'

Using TensorFlow backend.
Creating model, this may take a second...
2020-08-12 19:49:22.341740: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-12 19:49:22.383002: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-12 19:49:22.383441: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-08-12 19:49:22.383649: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-08-12 19:49:22.385074: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.s

# Restart full model training

In [32]:
last_model = 'aerialnet_project/11_classes/snapshots/resnet50_csv_53.h5'

!python keras-retinanet/keras_retinanet/bin/train.py \
--config aerialnet_project/11_classes/config.ini \
--random-transform \
--weights {last_model} \
--initial-epoch 53 \
--weighted-average \
--batch-size 4 \
--steps 574 \
--epochs 100 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir aerialnet_project/11_classes/logs \
--snapshot-path aerialnet_project/11_classes/snapshots \
--validation-freq 5 \
--lr 1e-5 \
--reduce-lr-patience 2 \
--reduce-lr-factor 0.1 \
--gamma 3.0 \
csv 'aerialnet_project/11_classes/data/11_classes_dataset_train.csv' 'aerialnet_project/11_classes/data/classes.csv' \
--val-annotations 'aerialnet_project/11_classes/data/11_classes_dataset_test.csv'

Using TensorFlow backend.
Creating model, this may take a second...
2020-08-12 22:12:48.546654: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-12 22:12:48.578359: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-12 22:12:48.578657: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-08-12 22:12:48.578799: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-08-12 22:12:48.579737: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.s

# DEBUG

If the model is not learning or has low accuracy, check the dataset and the anchors of the model to determine if the problem lies in a mismatch between the dataset and the anchors.

In [201]:
!python keras-retinanet/keras_retinanet/bin/debug.py \
--config aerialnet/7_classes/config.ini \
--show-annotations \
csv 'aerialnet/7_classes/data/enap_dataset_train.csv' 'aerialnet/7_classes/data/classes.csv'

Using TensorFlow backend.
^C


In [84]:
%pwd

'/home/aikauel/enap'

# Evaluate model

In [15]:
# path to trained weights
last_model = 'aerialnet_project/11_classes/snapshots/resnet50_csv_08.h5'

# evaluate model on test.csv
!python keras-retinanet/keras_retinanet/bin/evaluate.py \
--config aerialnet_project/11_classes/config.ini \
--image-min-side 1000 \
--image-max-side 1000 \
--max-detections 300 \
--save-path aerialnet_project/11_classes/data/output \
csv 'aerialnet_project/11_classes/data/11_classes_dataset_test.csv' \
'aerialnet_project/11_classes/data/classes.csv' \
{last_model} --convert-model

#%cd ..

Using TensorFlow backend.
Loading model, this may take a second...
2020-08-12 16:12:58.533680: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-12 16:12:58.567577: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-12 16:12:58.567893: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-08-12 16:12:58.568043: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-08-12 16:12:58.568999: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so

## Convert training model to inference model

In [52]:
os.getcwd()

'/home/aikauel/enap'

In [34]:
# path to trained weights
last_model = 'aerialnet_project/11_classes/snapshots/resnet50_csv_65.h5'

# convert trained model to inference model to be used for generating predictions
inference_model = last_model.replace('snapshots/', 'inference/')
!python keras-retinanet/keras_retinanet/bin/convert_model.py {last_model} {inference_model} \
--config aerialnet_project/11_classes/config.ini

Using TensorFlow backend.
2020-08-12 23:26:27.908836: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-12 23:26:27.940081: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-12 23:26:27.940421: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-08-12 23:26:27.940589: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-08-12 23:26:27.941504: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-08-12 23:26:27.942526: I tensorf

## Test model

In [44]:
!python aerialnet_project/shared_utils/image_inference_write.py \
-n True \
-i 'data/test_images/' \
-t 0.53 \
-m {inference_model} \
-o aerialnet_project/11_classes/data/test_images_output \
-l aerialnet_project/11_classes/data/classes.csv

Using TensorFlow backend.
2020-08-13 00:25:24.618493: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-13 00:25:24.649844: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-13 00:25:24.650183: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-08-13 00:25:24.650352: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-08-13 00:25:24.651439: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-08-13 00:25:24.652558: I tensorf

In [24]:
last_model = 'aerialnet_project/11_classes/snapshots/resnet50_csv_28.h5'
inference_model = last_model.replace('snapshots/', 'inference/')

In [35]:
inference_model

'aerialnet_project/11_classes/inference/resnet50_csv_65.h5'

# Test model on images without annotations (unseen in train-val-test)

In [2]:
# model selection
last_model = 'aerialnet_project/11_classes/snapshots/resnet50_csv_65.h5'
inference_model = last_model.replace('snapshots/', 'inference/')

In [3]:
# test dataset selection
test_dataset = 'data/test_datasets/week_10-14_08_2020/'
output_path = 'aerialnet_project/11_classes/data/test_datasets/week_10-14_08_2020'

In [4]:
'''!python aerialnet_project/shared_utils/image_inference_write.py \
-n True \
-i 'data/test_images/' \
-t 0.425 \
-m {inference_model} \
-o aerialnet_project/11_classes/data/test_images_output \
-l aerialnet_project/11_classes/data/classes.csv'''

!python aerialnet_project/shared_utils/image_inference_write.py \
-n True \
-i {test_dataset} \
-t 0.53 \
-m {inference_model} \
-o {output_path} \
-l aerialnet_project/11_classes/data/classes.csv \
-s True

Using TensorFlow backend.
2020-08-14 12:14:24.774058: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-14 12:14:24.808881: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-14 12:14:24.809207: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-08-14 12:14:24.809372: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-08-14 12:14:24.810589: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-08-14 12:14:24.811679: I tensorf

## Generate detections on test images - IMAGES

In [46]:
os.getcwd()

'/home/aikauel/enap'

In [50]:
inference_model

'snapshots/resnet50_csv_30_inference.h5'

In [308]:
# create output directory where you want to save images with bounding boxes
#!mkdir /content/data/output

# generate detections on images
!python ije_retinanet/image_inference_print.py \
-i 'data/test_images' \
-t 0.6 \
-m  {inference_model} \
-o data/output

Using TensorFlow backend.
2020-07-30 21:25:18.337695: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-30 21:25:18.367392: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-30 21:25:18.367760: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-07-30 21:25:18.367915: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-30 21:25:18.368876: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-07-30 21:25:18.369854: I tensorf

/home/aikauel/anaconda3/envs/enap/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
[FINAL] task completed!
